In [15]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
import json
import os
import subprocess
import sys
import datetime
from typing import Any, Dict, List, Optional, Type, Union
from datetime import datetime
import pandas as pd
import os
import sys
import numpy as np
import importlib
from concurrent.futures import ThreadPoolExecutor

from pys.data_collection.bonds_parser import MOEXBondHistoricalParser


In [3]:
bond_parser = MOEXBondHistoricalParser()

result = bond_parser.parse_from_2024_to_now(
    include_non_trading_days=True,  # Включаем выходные дни
    analyze_continuity=True,        # Анализируем непрерывность и создаем полный датасет
    yield_range=(0, 100),           # Широкий диапазон - берем все
    price_range=(0, 100),           # Широкий диапазон - берем все
    duration_range=(0, 7200)         # Широкий диапазон - берем все
)

print(f"Всего дней с данными: {len(result)}")
trading_days = sum(1 for date in result.keys() if datetime.strptime(date, '%Y-%m-%d').weekday() < 5)
print(f"Торговых дней: {trading_days}")
print(f"Нерабочих дней с данными: {len(result) - trading_days}")



2025-04-25 17:38:34 - MOEXBondHistoricalParser - INFO - MOEX Bond Parser initialized
2025-04-25 17:38:34 - MOEXBondHistoricalParser - INFO - Запуск массового парсинга данных с 2024-01-01 по 2025-04-24
2025-04-25 17:38:34 - MOEXBondHistoricalParser - INFO - Запрошен период с 2024-01-01 по 2025-04-24
2025-04-25 17:38:34 - MOEXBondHistoricalParser - INFO - Получение облигаций за период с 2024-01-01 по 2025-04-24
2025-04-25 17:38:34 - MOEXBondHistoricalParser - INFO - Обработка даты: 2024-01-01 (рабочий день)
2025-04-25 17:38:34 - MOEXBondHistoricalParser - INFO - Получение списка облигаций на 2024-01-01


KeyboardInterrupt: 

In [4]:
optimal_threshold = bond_parser.find_optimal_threshold(
    start_date=datetime(2024, 1, 1),
    end_date=datetime(2025, 4, 22),
    min_bonds=20,
    max_threshold=99
)

print(f"Оптимальный порог непрерывности: {optimal_threshold}%")

dataset = bond_parser.generate_dataset_with_threshold(
    start_date=datetime(2024, 1, 1),
    end_date=datetime(2025, 4, 22),
    continuity_threshold=optimal_threshold
)

if dataset is not None:
    print(f"Создан датасет с {len(dataset['secid'].unique())} облигациями и {len(dataset['date'].unique())} датами")
else:
    print("Не удалось создать датасет")

2025-04-25 17:38:44 - MOEXBondHistoricalParser - INFO - Поиск оптимального порога непрерывности для периода с 2024-01-01 по 2025-04-22
2025-04-25 17:38:44 - MOEXBondHistoricalParser - INFO - Анализ непрерывности данных по облигациям с 2024-01-01 по 2025-04-22
2025-04-25 17:38:44 - MOEXBondHistoricalParser - INFO - Найдено 342 рабочих дней в указанном периоде
2025-04-25 17:38:44 - MOEXBondHistoricalParser - INFO - Найдено 340 дней с доступными данными


2025-04-25 17:38:44 - MOEXBondHistoricalParser - INFO - Всего уникальных облигаций: 355
2025-04-25 17:38:44 - MOEXBondHistoricalParser - INFO - Облигаций с непрерывными данными: 0 (0.00%)
2025-04-25 17:38:44 - MOEXBondHistoricalParser - INFO - Результаты анализа сохранены в /Users/aeshef/Documents/GitHub/kursach/data/processed_data/BONDS/moex/continuity_analysis_20240101_20250422
2025-04-25 17:38:44 - MOEXBondHistoricalParser - INFO - При пороге 99%: доступно 49 облигаций
2025-04-25 17:38:44 - MOEXBondHistoricalParser - INFO - Найден оптимальный порог: 99%
2025-04-25 17:38:44 - MOEXBondHistoricalParser - INFO - Генерация датасета с порогом непрерывности 99% за период с 2024-01-01 по 2025-04-22
2025-04-25 17:38:44 - MOEXBondHistoricalParser - INFO - Анализ непрерывности данных по облигациям с 2024-01-01 по 2025-04-22
2025-04-25 17:38:44 - MOEXBondHistoricalParser - INFO - Найдено 342 рабочих дней в указанном периоде
2025-04-25 17:38:44 - MOEXBondHistoricalParser - INFO - Найдено 340 дне

Оптимальный порог непрерывности: 99%


2025-04-25 17:38:44 - MOEXBondHistoricalParser - INFO - Всего уникальных облигаций: 355
2025-04-25 17:38:44 - MOEXBondHistoricalParser - INFO - Облигаций с непрерывными данными: 0 (0.00%)
2025-04-25 17:38:44 - MOEXBondHistoricalParser - INFO - Найдено 49 облигаций с порогом непрерывности не менее 99%
2025-04-25 17:38:45 - MOEXBondHistoricalParser - INFO - Датасет с порогом непрерывности 99% создан и сохранен в /Users/aeshef/Documents/GitHub/kursach/data/processed_data/BONDS/moex/threshold_dataset_20240101_20250422_99
2025-04-25 17:38:45 - MOEXBondHistoricalParser - INFO - Всего дат: 340, облигаций: 49


Создан датасет с 49 облигациями и 339 датами


In [16]:
import sys
import importlib

from pys.data_collection.private_info import BASE_PATH
from pys.data_collection.bonds_processor import run_pipeline_bonds_processor

In [17]:
results = run_pipeline_bonds_processor(
    base_path=BASE_PATH,
    # Дополнительные параметры можно указать при необходимости
    # min_yield=7.0,
    # max_duration=4.0,
    # n_bonds=7
)


2025-04-25 17:53:17 - BondsProcessor - INFO - Loaded previous recommendations from 2025-04-25
2025-04-25 17:53:17 - BondsProcessor - INFO - Loading direct data from /Users/aeshef/Documents/GitHub/kursach/data/processed_data/BONDS/moex/threshold_dataset_20240101_20250422_99/bonds_dataset.csv
2025-04-25 17:53:17 - BondsProcessor - INFO - Renamed columns: {'secid': 'security_code', 'name': 'full_name', 'price': 'price_pct', 'duration': 'duration_months', 'maturity_date': 'expiration_date', 'volume': 'trading_volume', 'value': 'trading_value'}
2025-04-25 17:53:17 - BondsProcessor - INFO - Created duration_years column from duration_months
2025-04-25 17:53:17 - BondsProcessor - INFO - Created snapshot_date from date column
2025-04-25 17:53:17 - BondsProcessor - INFO - Filtered data to use only latest snapshot: 2025-04-22 00:00:00
2025-04-25 17:53:17 - BondsProcessor - INFO - Starting filters with 49 bonds
2025-04-25 17:53:17 - BondsProcessor - INFO - After min_yield filter: 14 bonds
2025-04

Обработано 10 облигаций

=== Рекомендованный облигационный портфель ===
      security_code   full_name  yield  duration_years  weight
16726  RU000A0JUAH8  РЖД-30 обл  20.63        0.510833  0.3305
16696  KG000A3LSJ06  Кыргыз. 02  20.77        0.658333  0.2565
16709  RU000A0JSGV0  РЖД-32 обл  20.96        1.135833  0.1487
16712  RU000A0JT6B2   ВЭБ.РФ 19  19.91        0.922500  0.1830
16711  RU000A0JT403   ВЭБ.РФ 18  20.11        2.077500  0.0813

Портфель сохранен в /Users/aeshef/Documents/GitHub/kursach/data/processed_data/BONDS/results/bond_portfolio_20250425.csv
Визуализация сохранена в /Users/aeshef/Documents/GitHub/kursach/data/processed_data/BONDS/results/bond_universe_20250425.png


2025-04-25 17:53:18 - BondsProcessor - INFO - Portfolio analysis generated successfully



=== Характеристики портфеля ===
Средневзвешенная доходность: 20.54%
Средневзвешенная дюрация: 0.84 лет
Уровень риска (дюрация): Низкий

Анализ успешно завершен!
